In [1]:
import argparse

import torch
import uuid

import numpy as np

from torch_geometric.nn import Node2Vec
from torch_geometric.utils import erdos_renyi_graph

import os
import csv

import gph
import gtda
from gtda import diagrams
from gtda import homology

import matplotlib.pyplot as plt

import sys
# appending a path
sys.path.append('../rawos')
import metrics


In [2]:

def get_dimension(diagrams, dim=0):
    """Get specific dimension of persistence diagram."""
    # This reads a little bit weird because of the idiosyncratic way of
    # handling masks. We first extract all triples in all time steps
    # that match the specified dimension. *Then* we remove the dimension
    # information (since it is spurious) and reshape the array to
    # account for the original number of time steps again. Whew!
    mask = diagrams[..., 2] == dim
    diagrams = diagrams[mask][:, :2]
    return diagrams

In [12]:
def compute_total_persistence(filename, max_dim): 
    """takes as input an embedding in the file called filename 
    and returns a vector of the total persistence in each dim"""
    
    emb = np.loadtxt(filename, delimiter="\t")
    diam = metrics.diameter(emb)
    emb= emb/diam 
    ph = gtda.homology.VietorisRipsPersistence(metric='euclidean', max_edge_length=np.inf, homology_dimensions=tuple(range(max_dim+1)), coeff=2, infinity_values=None, n_jobs=None).fit_transform([emb])
    
    total_pers=np.zeros((max_dim+1,1))
    for i in range(max_dim+1): 
        dgm_i= get_dimension(ph[0,:,:],i)
        total_pers[i]= total_persistence(dgm_i)
        
    return total_pers
    
    
    
    

In [13]:
filename ='../results/er-c5-d16-l5-n10-p0_25-1de110b3e022445bbcffe280ac0601a8.tsv' 
compute_total_persistence(filename, 1)

array([[10.14294864],
       [ 0.01030475]])

In [5]:
## import files
c = 5
d = 16
l = 5
n = 10

max_dim = 1

"""c = 5
d= 5
l=10
n=10"""



filename = 'er'
filename += '-c'+str(c)
filename += '-d'+ str(d)
filename += '-l'+str(l)
filename += '-n'+ str(n)

## get filenames according to filter
path = '../results'
#path = '../rawos'
files = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and filename in i:
        files.append(i)

## load embeddings
Emb_list= []

for i,file in zip(range(len(files)),files): 
    emb = np.loadtxt(path+'/'+file, delimiter="\t")
    diam = metrics.diameter(emb)
    Emb_list.append(emb/diam)
    
ph = gtda.homology.VietorisRipsPersistence(metric='euclidean', max_edge_length=np.inf, homology_dimensions=tuple(range(max_dim+1)), coeff=2, infinity_values=None, n_jobs=None).fit_transform(Embs)


200
200
200
200
200
200
200
200
200
200
